# LCA with foreground

This notebook demonstrates how to include foregrounds system into sensitivity analysis. The background database is manipulated as matrix, so as the foreground system. To run the LCA simulation, the library designed to place the foreground system in the right left corner.

### Preparation

1. Direct to the correct Brightway project and ensure the background database is loaded.

In [1]:
import bw2data as bd

bd.projects.set_current("advlca25")
bd.databases

Databases dictionary with 4 object(s):
	ALIGNED-biob-prod-dummy
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-consequential
	exldb

2. Importing required libraries

In [2]:
import os
import sys
sys.path.append(os.path.abspath(".."))
from bw_bamboo.background_importer import *
from bw_bamboo.foreground_importer import *
from bw_bamboo.datapackage_builder import *
from bw_bamboo.uncertainty_handler import *
from bw_bamboo.lca_wrapper import *
from bw_bamboo.uncertainty_importer import *

3. Define some constants

In [3]:
GHG = ["CO2 - combustion - air",
        "CO2 - non combustion - Cement production - air",
        "CO2 - non combustion - Lime production - air",
        "CO2 - waste - fossil - air",
        "CH4 - agriculture - air",
        "CH4 - waste - air",
        "CH4 - combustion - air",
        "CH4 - non combustion - Extraction/production of (natural) gas - air",
        "CH4 - non combustion - Extraction/production of crude oil - air",
        "CH4 - non combustion - Mining of antracite - air",
        "CH4 - non combustion - Mining of bituminous coal - air",
        "CH4 - non combustion - Mining of coking coal - air",
        "CH4 - non combustion - Mining of lignite (brown coal) - air",
        "CH4 - non combustion - Mining of sub-bituminous coal - air",
        "CH4 - non combustion - Oil refinery - air",
        "N2O - combustion - air",
        "N2O - agriculture - air",
        "SF6 - air"]

METHOD = ('ecoinvent-3.11', 'IPCC 2013', 'climate change', 'global temperature change potential (GTP100)')

# BACKGROUND DATABASE FILE PATH
EXIOBASE_AGGREGATED_A_FILE = os.path.join(os.getcwd(), "data/A.txt")
EXIOBASE_AGGREGATED_S_FILE = os.path.join(os.getcwd(), "data/S.txt")

# FOREGROUND DATABASE FILE PATH
FOREGROUND_1 = os.path.join(os.getcwd(), "data/foreground_system_1.csv")
FOREGROUND_2 = os.path.join(os.getcwd(), "data/foreground_system_2.csv")

# CHARACTERIZATION FACTOR MAPPING FILE PATH
CF_MAPPING_FILE = os.path.join(os.getcwd(), "data/cf_mapping_file.csv")

### Step 1: Get the background database matrices

1. Get technosphere matrix

In [4]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
tech_matrix = bg_importer.build_tech_matrix(raw_tech)

print(f"The shape of technosphere matrix is: {tech_matrix.shape}")

The shape of technosphere matrix is: (76, 76)


2. Get biosphere matrix

In [5]:
bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bio_matrix = bg_importer.build_bio_matrix(bio_df, GHG)

print(f"The shape of biosphere matrix is: {bio_matrix.shape}")

The shape of biosphere matrix is: (18, 76)


3. Get characterization factor matrix

In [6]:
# get characterization factor matrix from code.
cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG, "ecoinvent-3.11-biosphere", METHOD, source="code")

# print the diagonal to check the values.
print(f"The diagonal values of characterization factor matrix: \n {cf_matrix.diagonal()}")

All characterization factors have been found.
The diagonal values of characterization factor matrix: 
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 4.30000000e+00 5.70000000e+00 5.70000000e+00 5.70000000e+00
 5.70000000e+00 5.70000000e+00 5.70000000e+00 5.70000000e+00
 5.70000000e+00 5.70000000e+00 5.70000000e+00 2.34200000e+02
 2.34200000e+02 2.82148093e+04]


In [7]:
# get characterization factor matrix from the file directly.
cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG)  # By default, source="cf", so you can omit some parameters.

# print the diagonal to check the values.
print(f"The diagonal values of characterization factor matrix: \n {cf_matrix.diagonal()}")

All characterization factors have been found.
The diagonal values of characterization factor matrix: 
 [1.00e+00 1.00e+00 1.00e+00 1.00e+00 2.70e+01 2.98e+01 2.98e+01 2.98e+01
 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.73e+02
 2.73e+02 2.52e+04]


### Step 2: Import the foreground database

4. Check out the foreground data

In [8]:
fg_tech_df = pd.read_table(FOREGROUND_2, sep=',')
fg_tech_df.head()

,Activity name,Exchange name,Exchange type,Exchange amount,Exchange uncertainty type,GSD,Exchange negative
0,column_1,EU28-Agriculture-Forestry-Fishing,technosphere,0.000000,0,NaN,NaN
1,column_1,EU28-Energy,technosphere,NaN,0,NaN,NaN
2,column_1,EU28-Natural gas and services related to natur...,technosphere,NaN,0,NaN,NaN
3,column_1,EU28-Industry,technosphere,0.000001,2,3.330005,False
4,column_1,EU28-Motor Gasoline,technosphere,-0.000016,2,1.540000,True


5. Get activities in foreground system.

In [9]:
fg_activities = get_fg_activities(FOREGROUND_2, ",")
fg_activities

['column_1', 'column_2']

6. Get activities in background system.

In [10]:
bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
bg_activities[:5] # only list the first five activities here

['EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
 'EU28-Industry',
 'EU28-Motor Gasoline']

7. Get foreground system matrices

In [11]:
fg_importer = ForegroundImporter()

fgbg, fgfg, bgfg, bifg = fg_importer.extend_matrix(fg_tech_df, GHG, fg_activities, bg_activities)

8. Concatenate foreground system with background system

In [12]:
full_tech_matrix, full_bio_matrix = fg_importer.concatenate_matrix(tech_matrix, bio_matrix, fgbg, fgfg, bgfg, bifg)

# to compare the matrix before and after concatenate with foreground system
print(f"Technosphere shape change from {tech_matrix.shape} to {full_tech_matrix.shape}.")
print(f"Biosphere shape change from {bio_matrix.shape} to {full_bio_matrix.shape}.")

Technosphere shape change from (76, 76) to (78, 78).
Biosphere shape change from (18, 76) to (18, 78).


### Step 3: Build datapackage

9. Build datapackage

In [13]:
dp_builder = DatapackageBuilder()
datapackage_data = dp_builder.prepare_dp_matrix(full_tech_matrix, full_bio_matrix, cf_matrix)
datapackage = dp_builder.prepare_datapackage(datapackage_data)

### Step 4: Run the simulation

10. Run static simulation

In [14]:
mysector = 69  # RoW-Services

activities = fg_activities + bg_activities

lca = bc.LCA(
            demand={mysector : 1},
            data_objs=[datapackage],
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {lca.score, activities[mysector]}")


Brightway calculated lca score: (1111401.9185845114, 'RoW-Collected and purified water, distribution services of water (41)')


### Step 5: Build datapackage

2. With both foreground system (2 cols) and background system

In [16]:
from bw_bamboo.lca_wrapper import *

full_tech_matrix_manual = full_tech_matrix.copy()

np.fill_diagonal(full_tech_matrix_manual, -full_tech_matrix_manual.diagonal())
full_tech_matrix_manual = -full_tech_matrix_manual

rca = LCAWrapper()
rca.manual_lca(full_tech_matrix_manual, full_bio_matrix, cf_matrix, mysector)

1111401.9185845105